In [1]:
from premise import *
import bw2data, bw2io, bw2calc
bw2data.projects.set_current("ecoinvent-3.11-cutoff")

In [2]:
clear_inventory_cache()

Inventory cache cleared!


In [3]:
scenarios = [
    #{"model": "remind", "pathway": "SSP1-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP1-NPi", "year": 2025,},
    #{"model": "remind", "pathway": "SSP1-PkBudg650", "year": 2025,},
    #{"model": "remind", "pathway": "SSP1-PkBudg1000", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-PkBudg650", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-PkBudg1000", "year": 2025,},
    #{"model": "remind", "pathway": "SSP3-rollBack", "year": 2025,},
    #{"model": "remind", "pathway": "SSP3-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP3-NPi", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP3-PkBudg1000", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2025,},
    {"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2050,},
    #{"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2025,},

    #{"model": "image", "pathway": "SSP1-L", "year": 2025,},
    #{"model": "image", "pathway": "SSP1-Ma", "year": 2025,},
    #{"model": "image", "pathway": "SSP1-VLLO", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-L", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-M", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-VLHO", "year": 2025,},
    #{"model": "image", "pathway": "SSP5-H", "year": 2025,},
    #{"model": "image", "pathway": "SSP3-H", "year": 2025,},

    #{"model": "tiam-ucl", "pathway": "SSP2-Base", "year": 2050,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP19", "year": 2025,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP26", "year": 2025,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP45", "year": 2025,},
]


ndb = NewDatabase(
        scenarios = scenarios,
        source_db="ecoinvent-3.11-cutoff",
        source_version="3.11",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        biosphere_name="biosphere3"
)


Cannot find the IAM scenario file at /Users/romain/GitHub/premise/premise/data/iam_output_files/remind-eu_SSP2-PkBudg650. Will check online.
premise v.(2, 3, 0, 'dev1')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your Brightway project:                                   |
| from premise_gwp import add_premise_gwp                          |
| a

remind-eu_SSP2-PkBudg650.csv: 19.8MB [00:05, 3.46MB/s]


remind-eu_SSP2-PkBudg650.csv downloaded successfully.
Reading remind-eu_SSP2-PkBudg650 as CSV file
Done!


In [4]:
ndb.update("metals")

Processing scenarios for sector 'metals': 100%|█| 1/1 [00:45<00:00, 45

Done!



In [5]:
ndb.write_db_to_brightway("test metals 3")

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:28


Title: Writing activities to SQLite3 database:
  Started: 08/26/2025 13:14:33
  Finished: 08/26/2025 13:15:01
  Total time elapsed: 00:00:28
  CPU %: 92.00
  Memory %: 10.48
Created database: test metals 3
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev/export/change reports/.


In [6]:
import bw2data
bw2data.projects.set_current("ecoinvent-3.11-cutoff")
db = bw2data.Database("test metals 3")
 
markets_list = []
for ds in db:
    if ds["location"] == "World":
        markets_list.append(ds["name"])
 
dbs = [
    "ecoinvent-3.11-cutoff",
    "test metals 3"
]
fus = []
for d in dbs:
    for ds in bw2data.Database(d):
        if ds["name"] in markets_list and ds["location"] in ("GLO", "World"):
            if d == "test metals 3":
                if ds["location"] == "World":
                    fus.append(ds)
            else:
                fus.append(ds)
 
selected_methods = [m for m in bw2data.methods if "ef v3.1 no" in str(m).lower()]
 
import bw2calc
import numpy as np
from bw2calc.errors import OutsideTechnosphere
 
 
def get_lcia_scores(products, categories, results):
    lca = bw2calc.LCA({products[0]: 1}, categories[0])
    lca.lci()
    lca.lcia()
    method_matrices = [lca.characterization_matrix.copy()]
    for other_method in categories[1:]:
        # Only build each characterization matrix once instead of once per product
        lca.switch_method(other_method)
        method_matrices.append(lca.characterization_matrix.copy())
    for i, product in enumerate(products):
        print(product)
        try:
            lca.redo_lci({product: 1})
        except OutsideTechnosphere:
            lca = bw2calc.LCA({product: 1}, categories[0])
            lca.lci()
            lca.lcia()
 
            method_matrices = [lca.characterization_matrix.copy()]
            for other_method in categories[1:]:
                # Only build each characterization matrix once instead of once per product
                lca.switch_method(other_method)
                method_matrices.append(lca.characterization_matrix.copy())
        for j, characterization_matrix in enumerate(method_matrices):
            results[i, j] = (characterization_matrix * lca.inventory).sum()
    return results
 
results = np.zeros((len(fus), len(selected_methods)))
results = get_lcia_scores(fus, selected_methods, results)
 
 
import pandas as pd
df = pd.DataFrame(columns=[e[1] for e in selected_methods], index=[x["name"] for x in fus], data=results)
 
df["database"] = [x["database"] for x in fus]
df["location"] = [x["location"] for x in fus]
 
# Step 1: Copy dataframe
df_rel = df.copy()
 
# Identify columns with impact scores (exclude identifiers)
id_cols = [ "database", "location"]
impact_cols = [c for c in df.columns if c not in id_cols]
 
# Step 2: Normalize relative to ecoinvent-3.11-cutoff (index 100)
df_rel_list = []
for metal, group in df.groupby(df.index):
    ref = group[group["database"] == "ecoinvent-3.11-cutoff"][impact_cols]
    if ref.empty:
        continue
    ref_values = ref.iloc[0]
    # Compute relative values
    temp = group.copy()
    for col in impact_cols:
        temp[col] = (temp[col] / ref_values[col]) * 100 if ref_values[col] != 0 else None
    df_rel_list.append(temp)
 
df_rel = pd.concat(df_rel_list, ignore_index=False)
 
# Step 3: Save to new Excel file
out_path = "comparison_metals_relative.xlsx"
df_rel.to_excel(out_path, index=True)

'market for erbium oxide' (kilogram, GLO, None)
'market for titanium' (kilogram, GLO, None)
'market for tungsten concentrate' (kilogram, GLO, None)
'market for copper, cathode' (kilogram, GLO, None)
'market for sulfur' (kilogram, GLO, None)
'market for cobalt' (kilogram, GLO, None)
'market for chromite ore concentrate' (kilogram, GLO, None)
'market for ilmenite, 54% titanium dioxide' (kilogram, GLO, None)
'market for aluminium, primary, liquid' (kilogram, GLO, None)
'market for boron carbide' (kilogram, GLO, None)
'market for ytterbium oxide' (kilogram, GLO, None)
'market for uranium ore, as U' (kilogram, GLO, None)
'market for beryllium' (kilogram, GLO, None)
'market for silicon, metallurgical grade' (kilogram, GLO, None)
'market for gold' (kilogram, GLO, None)
'market for sodium borates' (kilogram, GLO, None)
'market for copper, anode' (kilogram, GLO, None)
'market for spodumene' (kilogram, GLO, None)
'market for praseodymium oxide' (kilogram, GLO, None)
'market for lead concentrate'